## Scenario 2: A cross-functional team with one data scientist working on an ML model

MLflow setup:

tracking server: yes, local server
backend store: sqlite database
artifacts store: local filesystem
The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

mlflow server --backend-store-uri sqlite:///backend.db

In [12]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [13]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [14]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1723604851972, experiment_id='0', last_update_time=1723604851972, lifecycle_stage='active', name='Default', tags={}>]

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/08/14 03:22:34 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:17: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warni

default artifacts URI: '/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts_local/1/6cea9ec61b7b46cf824487fefbe3eb60/artifacts'


In [16]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts_local/1', creation_time=1723605754094, experiment_id='1', last_update_time=1723605754094, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1723604851972, experiment_id='0', last_update_time=1723604851972, lifecycle_stage='active', name='Default', tags={}>]

## Interacting with the model registry

In [17]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [18]:
client.search_registered_models()

[]

In [21]:
runs = client.search_runs(experiment_ids=['1'], max_results=1)
if runs:
    run_id = runs[0].info.run_id
else:
    raise Exception("No runs found in the specified experiment.")


model_uri = f"runs:/{run_id}/models"
model_name = 'iris-classifier'

client.create_registered_model(name=model_name)
client.create_model_version(
    name=model_name,
    source=model_uri,
    run_id=run_id
)

2024/08/14 03:26:18 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1


<ModelVersion: aliases=[], creation_timestamp=1723605978877, current_stage='None', description='', last_updated_timestamp=1723605978877, name='iris-classifier', run_id='6cea9ec61b7b46cf824487fefbe3eb60', run_link='', source='runs:/6cea9ec61b7b46cf824487fefbe3eb60/models', status='READY', status_message='', tags={}, user_id='', version='1'>